# Landmark Search Agent Tutorial - Priority 1 Implementation

This notebook demonstrates the Agent Catalog landmark search agent using LlamaIndex with Couchbase vector store and Arize Phoenix evaluation. Uses Priority 1 AI services with standard OpenAI wrappers and Capella (simple & fast).


## Setup and Imports

Import all necessary modules for the landmark search agent using self-contained setup.


In [1]:
import base64
import getpass
import httpx
import json
import logging
import os
import sys
import time
from datetime import timedelta

import agentc
import dotenv
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import BucketType, CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.nvidia import NVIDIA
from llama_index.llms.openai_like import OpenAILike
from llama_index.vector_stores.couchbase import CouchbaseSearchVectorStore
from pydantic import SecretStr

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Reduce noise from various libraries during embedding/vector operations
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)

# Set default values for travel-sample bucket configuration
DEFAULT_BUCKET = "travel-sample"
DEFAULT_SCOPE = "agentc_data"
DEFAULT_COLLECTION = "landmark_data"
DEFAULT_INDEX = "landmark_data_index"
DEFAULT_CAPELLA_API_EMBEDDING_MODEL = "Snowflake/snowflake-arctic-embed-l-v2.0"
DEFAULT_CAPELLA_API_LLM_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
DEFAULT_NVIDIA_API_LLM_MODEL = "meta/llama-3.1-70b-instruct"


## Self-Contained Setup Functions

Define all necessary setup functions inline for a self-contained notebook.


In [2]:
def setup_environment():
    """Setup default environment variables for agent operations."""
    defaults = {
        "CB_BUCKET": "travel-sample",
        "CB_SCOPE": "agentc_data",
        "CB_COLLECTION": "landmark_data",
        "CB_INDEX": "landmark_data_index",
        "NVIDIA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "NVIDIA_API_LLM_MODEL": "meta/llama-3.1-70b-instruct",
        "CAPELLA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "CAPELLA_API_LLM_MODEL": "meta-llama/Llama-3.1-8B-Instruct",
    }
    
    for key, value in defaults.items():
        if not os.getenv(key):
            os.environ[key] = value
    
    logger.info("✅ Environment variables configured")


def test_capella_connectivity(api_key: str = None, endpoint: str = None) -> bool:
    """Test connectivity to Capella AI services."""
    try:
        test_key = api_key or os.getenv("CAPELLA_API_EMBEDDINGS_KEY") or os.getenv("CAPELLA_API_LLM_KEY")
        test_endpoint = endpoint or os.getenv("CAPELLA_API_ENDPOINT")
        
        if not test_key or not test_endpoint:
            return False
        
        # Simple connectivity test
        headers = {"Authorization": f"Bearer {test_key}"}
        
        with httpx.Client(timeout=10.0) as client:
            response = client.get(f"{test_endpoint.rstrip('/')}/v1/models", headers=headers)
            return response.status_code < 500
    except Exception as e:
        logger.warning(f"⚠️ Capella connectivity test failed: {e}")
        return False


def setup_ai_services(framework: str = "llamaindex", temperature: float = 0.0, application_span=None):
    """Priority 1: Capella AI with OpenAI wrappers (simple & fast) for LlamaIndex."""
    embeddings = None
    llm = None
    
    logger.info(f"🔧 Setting up Priority 1 AI services for {framework} framework...")
    
    # Priority 1: Capella AI with direct API keys and OpenAI wrappers
    if not embeddings and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_EMBEDDINGS_KEY"):
        try:
            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            api_key = os.getenv("CAPELLA_API_EMBEDDINGS_KEY")
            model = os.getenv("CAPELLA_API_EMBEDDING_MODEL")
            
            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                api_base = endpoint
            else:
                api_base = f"{endpoint}/v1"
            
            # Debug logging - same pattern as working test
            logger.info(f"🔧 Endpoint: {endpoint}")
            logger.info(f"🔧 Model: {model}")
            logger.info(f"🔧 API Base: {api_base}")
            
            embeddings = OpenAIEmbedding(
                api_key=api_key,
                api_base=api_base,
                model_name=model,
                embed_batch_size=30,
                # Note: LlamaIndex doesn't need check_embedding_ctx_length=False
            )
            logger.info("✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI embeddings failed: {type(e).__name__}: {e}")
    
    if not llm and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_LLM_KEY"):
        try:
            endpoint = os.getenv("CAPELLA_API_ENDPOINT")
            llm_key = os.getenv("CAPELLA_API_LLM_KEY")
            llm_model = os.getenv("CAPELLA_API_LLM_MODEL")
            
            # Handle endpoint that may or may not already have /v1 suffix
            if endpoint.endswith('/v1'):
                api_base = endpoint
            else:
                api_base = f"{endpoint}/v1"
            
            # Debug logging
            logger.info(f"🔧 LLM Endpoint: {endpoint}")
            logger.info(f"🔧 LLM Model: {llm_model}")
            logger.info(f"🔧 LLM API Base: {api_base}")
            
            llm = OpenAILike(
                model=llm_model,
                api_base=api_base,
                api_key=llm_key,
                is_chat_model=True,
                is_function_calling_model=False,  # KEY FIX - prevents 500 errors
                context_window=128000,  # Add context window for compatibility
                temperature=temperature,
                max_retries=1,  # Faster debugging
            )
            # Test the LLM works
            test_response = llm.complete("Hello")
            logger.info("✅ Using Priority 1: Capella AI LLM (OpenAI wrapper)")
        except Exception as e:
            logger.error(f"❌ Priority 1 Capella AI LLM failed: {type(e).__name__}: {e}")
            llm = None
    
    # Fallback: OpenAI
    if not embeddings and os.getenv("OPENAI_API_KEY"):
        try:
            embeddings = OpenAIEmbedding(
                model_name="text-embedding-3-small",
                api_key=os.getenv("OPENAI_API_KEY"),
            )
            logger.info("✅ Using OpenAI embeddings fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI embeddings failed: {e}")
    
    if not llm and os.getenv("OPENAI_API_KEY"):
        try:
            llm = OpenAILike(
                model="gpt-4o",
                api_key=os.getenv("OPENAI_API_KEY"),
                is_chat_model=True,
                is_function_calling_model=False,
                temperature=temperature,
            )
            logger.info("✅ Using OpenAI LLM fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI LLM failed: {e}")
    
    if not embeddings:
        raise ValueError("❌ No embeddings service could be initialized")
    if not llm:
        raise ValueError("❌ No LLM service could be initialized")
    
    logger.info(f"✅ Priority 1 AI services setup completed for {framework}")
    return embeddings, llm


# Setup environment
setup_environment()

# Test Capella AI connectivity if configured
if os.getenv("CAPELLA_API_ENDPOINT"):
    if not test_capella_connectivity():
        logger.warning("❌ Capella AI connectivity test failed. Will use fallback models.")
else:
    logger.info("ℹ️ Capella API not configured - will use fallback models")


2025-09-04 12:53:03,468 - INFO - ✅ Environment variables configured


## CouchbaseClient Class

Define the CouchbaseClient for all database operations and LlamaIndex agent creation.


In [3]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)

            # Use WAN profile for better timeout handling with remote clusters
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=20))
            logger.info("Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"Failed to connect to Couchbase: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str):
        """Setup collection - create scope and collection if they don't exist."""
        try:
            # Ensure cluster connection
            if not self.cluster:
                self.connect()

            # For travel-sample bucket, assume it exists
            if not self.bucket:
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"Connected to bucket '{self.bucket_name}'")

            # Setup scope
            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"Scope '{scope_name}' created successfully")

            # Setup collection - clear if exists, create if doesn't
            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists:
                logger.info(f"Collection '{collection_name}' exists, clearing data...")
                # Clear existing data
                self.clear_collection_data(scope_name, collection_name)
            else:
                logger.info(f"Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"Collection '{collection_name}' created successfully")

            time.sleep(3)

            # Create primary index
            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
                logger.info("Primary index created successfully")
            except Exception as e:
                logger.warning(f"Error creating primary index: {e}")

            logger.info("Collection setup complete")
            return self.bucket.scope(scope_name).collection(collection_name)

        except Exception as e:
            raise RuntimeError(f"Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str):
        """Clear all data from a collection."""
        try:
            logger.info(f"Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")

            # Use N1QL to delete all documents with explicit execution
            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)

            # Execute the query and get the results
            rows = list(result)

            # Wait a moment for the deletion to propagate
            time.sleep(2)

            # Verify collection is empty
            count_query = f"SELECT COUNT(*) as count FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            count_result = self.cluster.query(count_query)
            count_row = list(count_result)[0]
            remaining_count = count_row["count"]

            if remaining_count == 0:
                logger.info(f"Collection cleared successfully, {remaining_count} documents remaining")
            else:
                logger.warning(f"Collection clear incomplete, {remaining_count} documents remaining")

        except Exception as e:
            logger.warning(f"Error clearing collection data: {e}")
            # If N1QL fails, try to continue anyway
            pass

    def get_collection(self, scope_name: str, collection_name: str):
        """Get a collection object."""
        key = f"{scope_name}.{collection_name}"
        if key not in self._collections:
            self._collections[key] = self.bucket.scope(scope_name).collection(collection_name)
        return self._collections[key]

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index for the specified scope."""
        try:
            if not self.bucket:
                raise RuntimeError("Bucket not initialized. Call setup_collection first.")

            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"Vector search index '{index_name}' already exists")
        except Exception as e:
            raise RuntimeError(f"Error setting up vector search index: {e!s}")

    def load_landmark_data(self, scope_name, collection_name, index_name, embeddings):
        """Load landmark data into Couchbase."""
        try:
            # Import landmark data loading function
            # Use inline landmark data loading function (already defined in this notebook)
            # The function load_landmark_data_to_couchbase is defined inline above in this notebook

            # Load landmark data using the data loading script
            load_landmark_data_to_couchbase(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embeddings=embeddings,
                index_name=index_name,
            )
            logger.info("Landmark data loaded into vector store successfully")

        except Exception as e:
            raise RuntimeError(f"Error loading landmark data: {e!s}")

    def setup_vector_store_and_agent(self, catalog, span):
        """Setup vector store with landmark data and create agent."""
        # Setup AI services using Priority 1: Capella AI + OpenAI wrappers
        embeddings, llm = setup_ai_services(framework="llamaindex", temperature=0.1, application_span=span)
        
        # Set global LlamaIndex settings
        Settings.llm = llm
        Settings.embed_model = embeddings
        
        # Setup collection
        self.setup_collection(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"])
        
        # Setup vector search index - MUST have agentcatalog_index.json
        with open("agentcatalog_index.json") as file:
            index_definition = json.load(file)
        logger.info("Loaded vector search index definition from agentcatalog_index.json")
        self.setup_vector_search_index(index_definition, os.environ["CB_SCOPE"])
        
        # Load landmark data
        self.load_landmark_data(
            os.environ["CB_SCOPE"],
            os.environ["CB_COLLECTION"],
            os.environ["CB_INDEX"],
            embeddings,
        )
        
        # Create LlamaIndex ReAct agent
        agent = self.create_llamaindex_agent(catalog, span)
        
        return agent

    def create_llamaindex_agent(self, catalog, span):
        """Create LlamaIndex ReAct agent with landmark search tool from Agent Catalog."""
        try:
            # Get tools from Agent Catalog
            tools = []

            # Search landmarks tool
            search_tool_result = catalog.find("tool", name="search_landmarks")
            if search_tool_result:
                tools.append(
                    FunctionTool.from_defaults(
                        fn=search_tool_result.func,
                        name="search_landmarks",
                        description=getattr(search_tool_result.meta, "description", None)
                        or "Search for landmark information using semantic vector search. Use for finding attractions, monuments, museums, parks, and other points of interest.",
                    )
                )
                logger.info("Loaded search_landmarks tool from AgentC")

            if not tools:
                logger.warning("No tools found in Agent Catalog")
            else:
                logger.info(f"Loaded {len(tools)} tools from Agent Catalog")

            # Get prompt from Agent Catalog - REQUIRED, no fallbacks
            prompt_result = catalog.find("prompt", name="landmark_search_assistant")
            if not prompt_result:
                raise RuntimeError("Prompt 'landmark_search_assistant' not found in Agent Catalog")

            # Try different possible attributes for the prompt content
            system_prompt = (
                getattr(prompt_result, "content", None)
                or getattr(prompt_result, "template", None)
                or getattr(prompt_result, "text", None)
            )
            if not system_prompt:
                raise RuntimeError(
                    "Could not access prompt content from AgentC - prompt content is None or empty"
                )

            logger.info("Loaded system prompt from Agent Catalog")

            # Create ReAct agent with limits to prevent excessive iterations
            agent = ReActAgent.from_tools(
                tools=tools,
                llm=Settings.llm,
                verbose=True,
                system_prompt=system_prompt,
                max_iterations=12,
            )

            logger.info("LlamaIndex ReAct agent created successfully")
            return agent

        except Exception as e:
            raise RuntimeError(f"Error creating LlamaIndex agent: {e!s}")


## Standalone Agent Creation Function

Standalone version of the agent creation function for compatibility with main.py structure.


In [4]:
def create_llamaindex_agent(catalog, span):
    """Create LlamaIndex ReAct agent with landmark search tool from Agent Catalog."""
    try:
        from llama_index.core.agent import ReActAgent
        from llama_index.core.tools import FunctionTool

        # Get tools from Agent Catalog
        tools = []

        # Search landmarks tool
        search_tool_result = catalog.find("tool", name="search_landmarks")
        if search_tool_result:
            tools.append(
                FunctionTool.from_defaults(
                    fn=search_tool_result.func,
                    name="search_landmarks",
                    description=getattr(search_tool_result.meta, "description", None)
                    or "Search for landmark information using semantic vector search. Use for finding attractions, monuments, museums, parks, and other points of interest.",
                )
            )
            logger.info("Loaded search_landmarks tool from AgentC")

        if not tools:
            logger.warning("No tools found in Agent Catalog")
        else:
            logger.info(f"Loaded {len(tools)} tools from Agent Catalog")

        # Get prompt from Agent Catalog - REQUIRED, no fallbacks
        prompt_result = catalog.find("prompt", name="landmark_search_assistant")
        if not prompt_result:
            raise RuntimeError("Prompt 'landmark_search_assistant' not found in Agent Catalog")

        # Try different possible attributes for the prompt content
        system_prompt = (
            getattr(prompt_result, "content", None)
            or getattr(prompt_result, "template", None)
            or getattr(prompt_result, "text", None)
        )
        if not system_prompt:
            raise RuntimeError(
                "Could not access prompt content from AgentC - prompt content is None or empty"
            )

        logger.info("Loaded system prompt from Agent Catalog")

        # Create ReAct agent with reasonable iteration limit
        agent = ReActAgent.from_tools(
            tools=tools,
            llm=Settings.llm,
            verbose=True,  # Keep on for debugging
            system_prompt=system_prompt,
            max_iterations=10,  # Allow sufficient reasoning steps for complex landmark queries
        )

        logger.info("LlamaIndex ReAct agent created successfully")
        return agent

    except Exception as e:
        raise RuntimeError(f"Error creating LlamaIndex agent: {e!s}")


## Data Loading Module

Complete landmark data loading functions from data/landmark_data.py - inline for self-contained operation.


In [5]:
# Data loading functions from data/landmark_data.py
import couchbase.auth
import couchbase.cluster
import couchbase.exceptions
import couchbase.options
from llama_index.core import Document
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.couchbase import CouchbaseSearchVectorStore
from tqdm import tqdm


def get_cluster_connection():
    """Get a fresh cluster connection for each request."""
    try:
        auth = couchbase.auth.PasswordAuthenticator(
            username=os.environ["CB_USERNAME"],
            password=os.environ["CB_PASSWORD"],
        )
        options = couchbase.options.ClusterOptions(authenticator=auth)
        # Use WAN profile for better timeout handling with remote clusters
        options.apply_profile("wan_development")

        cluster = couchbase.cluster.Cluster(
            os.environ["CB_CONN_STRING"], options
        )
        cluster.wait_until_ready(timedelta(seconds=15))
        return cluster
    except couchbase.exceptions.CouchbaseException as e:
        logger.error(f"Could not connect to Couchbase cluster: {str(e)}")
        return None


def load_landmark_data_from_travel_sample():
    """Load landmark data from travel-sample.inventory.landmark collection."""
    try:
        cluster = get_cluster_connection()
        if not cluster:
            raise ConnectionError("Could not connect to Couchbase cluster")

        # Query to get all landmark documents from travel-sample.inventory.landmark
        query = """
        SELECT l.*, META(l).id as doc_id
        FROM `travel-sample`.inventory.landmark l
        ORDER BY l.name
        """

        logger.info("Loading landmark data from travel-sample.inventory.landmark...")
        result = cluster.query(query)

        landmarks = []
        logger.info("Processing landmark documents...")

        # Convert to list to get total count for progress bar
        landmark_rows = list(result)

        # Use tqdm for progress bar
        for row in tqdm(landmark_rows, desc="Loading landmarks", unit="landmarks"):
            landmark = row
            landmarks.append(landmark)

        logger.info(f"Loaded {len(landmarks)} landmarks from travel-sample.inventory.landmark")
        return landmarks

    except Exception as e:
        logger.error(f"Error loading landmark data: {str(e)}")
        raise


def get_landmark_texts():
    """Returns formatted landmark texts for vector store embedding from travel-sample data."""
    landmarks = load_landmark_data_from_travel_sample()
    landmark_texts = []

    logger.info("Generating landmark text embeddings...")

    # Use tqdm for progress bar while processing landmarks
    for landmark in tqdm(landmarks, desc="Processing landmarks", unit="landmarks"):
        # Start with basic info
        name = landmark.get("name", "Unknown Landmark")
        title = landmark.get("title", name)
        city = landmark.get("city", "Unknown City")
        country = landmark.get("country", "Unknown Country")

        # Build comprehensive text with all available fields
        text_parts = [f"{title} ({name}) in {city}, {country}"]

        # Add all fields dynamically instead of manual selection
        field_mappings = {
            "content": "Description",
            "address": "Address",
            "directions": "Directions",
            "phone": "Phone",
            "tollfree": "Toll-free",
            "email": "Email",
            "url": "Website",
            "hours": "Hours",
            "price": "Price",
            "activity": "Activity type",
            "type": "Type",
            "state": "State",
            "alt": "Alternative name",
            "image": "Image",
        }

        # Add all available fields
        for field, label in field_mappings.items():
            value = landmark.get(field)
            if value is not None and value != "" and value != "None":
                if isinstance(value, bool):
                    text_parts.append(f"{label}: {'Yes' if value else 'No'}")
                else:
                    text_parts.append(f"{label}: {value}")

        # Add geographic coordinates if available
        if landmark.get("geo"):
            geo = landmark["geo"]
            if geo.get("lat") and geo.get("lon"):
                accuracy = geo.get("accuracy", "Unknown")
                text_parts.append(f"Coordinates: {geo['lat']}, {geo['lon']} (accuracy: {accuracy})")

        # Add ID for reference
        if landmark.get("id"):
            text_parts.append(f"ID: {landmark['id']}")

        # Join all parts with ". "
        text = ". ".join(text_parts)
        landmark_texts.append(text)

    logger.info(f"Generated {len(landmark_texts)} landmark text embeddings")
    return landmark_texts


def load_landmark_data_to_couchbase(
    cluster, bucket_name: str, scope_name: str, collection_name: str, embeddings, index_name: str
):
    """Load landmark data from travel-sample into the target collection with embeddings."""
    try:
        # Check if data already exists
        count_query = (
            f"SELECT COUNT(*) as count FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
        )
        count_result = cluster.query(count_query)
        count_row = list(count_result)[0]
        existing_count = count_row["count"]

        if existing_count > 0:
            logger.info(
                f"Found {existing_count} existing documents in collection, skipping data load"
            )
            return

        # Get the source landmarks from travel-sample
        landmarks = load_landmark_data_from_travel_sample()
        landmark_texts = get_landmark_texts()

        # Setup vector store for the target collection
        vector_store = CouchbaseSearchVectorStore(
            cluster=cluster,
            bucket_name=bucket_name,
            scope_name=scope_name,
            collection_name=collection_name,
            index_name=index_name,
        )

        # Create LlamaIndex Documents
        logger.info(f"Creating {len(landmark_texts)} LlamaIndex Documents...")
        documents = []
        
        for i, (landmark, text) in enumerate(zip(landmarks, landmark_texts)):
            document = Document(
                text=text,
                metadata={
                    "landmark_id": landmark.get("id", f"landmark_{i}"),
                    "name": landmark.get("name", "Unknown"),
                    "city": landmark.get("city", "Unknown"),
                    "country": landmark.get("country", "Unknown"),
                    "activity": landmark.get("activity", ""),
                    "type": landmark.get("type", ""),
                    # Add the missing fields that search tool expects
                    "address": landmark.get("address", ""),
                    "phone": landmark.get("phone", ""),
                    "url": landmark.get("url", ""),
                    "hours": landmark.get("hours", ""),
                    "price": landmark.get("price", ""),
                    "state": landmark.get("state", ""),
                }
            )
            documents.append(document)

        # Use IngestionPipeline to process documents with embeddings
        logger.info(f"Processing documents with ingestion pipeline...")
        pipeline = IngestionPipeline(
            transformations=[SentenceSplitter(chunk_size=800, chunk_overlap=100), embeddings],
            vector_store=vector_store,
        )

        # Process documents in batches to avoid memory issues
        batch_size = 25  # Well below Capella AI embedding model limit
        total_batches = (len(documents) + batch_size - 1) // batch_size

        logger.info(f"Processing {len(documents)} documents in {total_batches} batches...")
        
        # Process in batches
        for i in tqdm(
            range(0, len(documents), batch_size),
            desc="Loading batches",
            unit="batch",
            total=total_batches,
        ):
            batch = documents[i : i + batch_size]
            pipeline.run(documents=batch)

        logger.info(
            f"Successfully loaded {len(documents)} landmark documents to vector store"
        )

    except Exception as e:
        logger.error(f"Error loading landmark data to Couchbase: {str(e)}")
        raise


def get_landmark_count():
    """Get the count of landmarks in travel-sample.inventory.landmark."""
    try:
        cluster = get_cluster_connection()
        if not cluster:
            raise ConnectionError("Could not connect to Couchbase cluster")

        query = "SELECT COUNT(*) as count FROM `travel-sample`.inventory.landmark"
        result = cluster.query(query)

        for row in result:
            return row["count"]

        return 0

    except Exception as e:
        logger.error(f"Error getting landmark count: {str(e)}")
        return 0


## Query Module

Complete query collections and functions from data/queries.py - inline for self-contained operation.


In [6]:
# Query functions and data from data/queries.py
from typing import Dict, List

# Landmark search queries (based on travel-sample data)
LANDMARK_SEARCH_QUERIES = [
    "Find museums and galleries in Glasgow",  # Art & Culture, Scotland
    "Show me restaurants serving Asian cuisine",  # Food & Dining, Real Asian restaurants
    "What attractions can I see in Glasgow?",  # General sightseeing, Scotland
    "Tell me about Monet's House",  # Specific landmark, France
    "Find places to eat in Gillingham",  # Food, Real UK town
]

# Comprehensive reference answers based on ACTUAL agent responses from travel-sample.inventory.landmark data
LANDMARK_REFERENCE_ANSWERS = [
    # Query 1: Glasgow museums and galleries
    """Glasgow has several museums and galleries including the Gallery of Modern Art (Glasgow) located at Royal Exchange Square with a terrific collection of recent paintings and sculptures, the Kelvingrove Art Gallery and Museum on Argyle Street with one of the finest civic collections in Europe including works by Van Gogh, Monet and Rembrandt, the Hunterian Museum and Art Gallery at University of Glasgow with a world famous Whistler collection, and the Riverside Museum at 100 Pointhouse Place with an excellent collection of vehicles and transport history. All offer free admission except for special exhibitions.""",

    # Query 2: Asian cuisine restaurants
    """There are several Asian restaurants available including Shangri-la Chinese Restaurant in Birmingham at 51 Station Street offering good quality Chinese food with spring rolls and sizzling steak, Taiwan Restaurant in San Francisco famous for their dumplings, Hong Kong Seafood Restaurant in San Francisco for sit-down dim sum, Cheung Hing Chinese Restaurant in San Francisco for Cantonese BBQ and roast duck, Vietnam Restaurant in San Francisco for Vietnamese dishes including crab soup and pork sandwich, and various other Chinese and Asian establishments across different locations.""",

    # Query 3: Glasgow attractions
    """Glasgow attractions include Glasgow Green (founded by Royal grant in 1450) with Nelson's Memorial and the Doulton Fountain, Glasgow University (founded 1451) with neo-Gothic architecture and commanding views, Glasgow Cathedral with fine Gothic architecture from medieval times, the City Chambers in George Square built in 1888 in Italian Renaissance style with guided tours available, Glasgow Central Station with its grand interior, and Kelvingrove Park which is popular with students and contains the Art Gallery and Museum.""",

    # Query 4: Monet's House
    """Monet's House is located in Giverny, France at 84 rue Claude Monet. The house is quietly eccentric and highly interesting in an Orient-influenced style, featuring Monet's collection of Japanese prints. The main attraction is the gardens around the house, including the water garden with the Japanese bridge, weeping willows and waterlilies which are now iconic. It's open April-October, Monday-Sunday 9:30-18:00, with admission €9 for adults, €5 for students, €4 for disabled visitors, and free for under-7s. E-tickets can be purchased online and wheelchair access is available.""",

    # Query 5: Gillingham restaurants
    """Gillingham has various dining options including Beijing Inn (Chinese restaurant at 3 King Street), Spice Court (Indian restaurant at 56-58 Balmoral Road opposite the railway station, award-winning with Sunday Buffet for £8.50), Hollywood Bowl (American-style restaurant at 4 High Street with burgers and ribs in a Hollywood-themed setting), Ossie's Fish and Chips (at 75 Richmond Road, known for the best fish and chips in the area), and Thai Won Mien (oriental restaurant at 59-61 High Street with noodles, duck and other oriental dishes).""",
]

# Create dictionary for backward compatibility
QUERY_REFERENCE_ANSWERS = {
    query: answer for query, answer in zip(LANDMARK_SEARCH_QUERIES, LANDMARK_REFERENCE_ANSWERS)
}

# Category-based queries for testing specific search capabilities (based on real data)
CATEGORY_QUERIES = {
    "cultural": [
        "Find museums and galleries in Glasgow",
        "Show me historic buildings and architecture",
        "What art collections can I visit?",
    ],
    "culinary": [
        "Show me restaurants serving Asian cuisine",
        "Find places to eat in Gillingham",
        "What dining options are available?",
    ],
    "sightseeing": [
        "What attractions can I see in Glasgow?",
        "Show me historic landmarks and buildings",
        "Find interesting places to visit",
    ],
    "specific": [
        "Tell me about Monet's House",
        "Show me the Glasgow Cathedral",
        "What can you tell me about the Burrell Collection?",
    ],
}

# Location-based queries for geographic diversity testing (based on real data)
LOCATION_QUERIES = {
    "Scotland": [
        "Find museums and galleries in Glasgow",
        "What attractions can I see in Glasgow?",
        "Show me historic buildings in Glasgow",
    ],
    "England": [
        "Find places to eat in Gillingham",
        "Show me restaurants serving Asian cuisine",
        "What landmarks are in Gillingham?",
    ],
    "France": [
        "Tell me about Monet's House",
        "Show me attractions in Giverny",
        "What can I visit in France?",
    ],
    "UK_General": [
        "Find attractions in the United Kingdom",
        "Show me places to visit in the UK",
        "What can I see in Britain?",
    ],
}

# Activity-based queries for testing different search patterns
ACTIVITY_QUERIES = [
    "What can I see in Glasgow?",  # 'see' activity queries
    "Where can I eat in Gillingham?",  # 'eat' activity queries
    "Show me places to dine",  # Generic eating queries
    "Find things to visit and see",  # Generic sightseeing queries
    "What museums can I visit?",  # Specific venue type queries
]


def get_all_queries() -> List[str]:
    """Get all queries for comprehensive testing."""
    all_queries = LANDMARK_SEARCH_QUERIES.copy()

    # Add category queries
    for category_list in CATEGORY_QUERIES.values():
        all_queries.extend(category_list)

    # Add location queries
    for location_list in LOCATION_QUERIES.values():
        all_queries.extend(location_list)

    # Add activity queries
    all_queries.extend(ACTIVITY_QUERIES)

    return all_queries


def get_reference_answer(query: str) -> str:
    """Get reference answer for a specific query."""
    return QUERY_REFERENCE_ANSWERS.get(query, "No reference answer available for this query.")


def get_queries_by_category(category: str) -> List[str]:
    """Get queries filtered by category."""
    if category == "basic":
        return LANDMARK_SEARCH_QUERIES
    elif category == "category":
        return [q for queries in CATEGORY_QUERIES.values() for q in queries]
    elif category == "location":
        return [q for queries in LOCATION_QUERIES.values() for q in queries]
    elif category == "activity":
        return ACTIVITY_QUERIES
    else:
        return get_all_queries()


def get_queries_for_evaluation(limit: int = 5) -> List[str]:
    """Get a subset of queries for evaluation purposes."""
    return LANDMARK_SEARCH_QUERIES[:limit]


## Landmark Search Agent Setup

Setup the complete landmark search agent infrastructure using LlamaIndex.


In [7]:
def setup_landmark_agent():
    """Setup the complete landmark search agent infrastructure and return the agent."""
    setup_environment()

    # Initialize Agent Catalog with credentials
    catalog = agentc.Catalog()
    span = catalog.Span(name="Landmark Search Agent Setup", blacklist=set())

    # Setup LLM and embeddings
    embeddings, llm = setup_ai_services(framework="llamaindex", temperature=0.1, application_span=span)

    # Set global LlamaIndex settings
    Settings.llm = llm
    Settings.embed_model = embeddings


    # Setup database client
    client = CouchbaseClient(
        conn_string=os.environ["CB_CONN_STRING"],
        username=os.environ["CB_USERNAME"],
        password=os.environ["CB_PASSWORD"],
        bucket_name=os.environ["CB_BUCKET"],
    )

    client.connect()

    # Setup vector store and agent
    agent = client.setup_vector_store_and_agent(catalog, span)

    return agent, client


# Inline evaluation templates for lenient evaluation
LENIENT_QA_PROMPT_TEMPLATE = """
You are an expert evaluator assessing if an AI assistant's response correctly answers the user's question about landmarks and attractions.

FOCUS ON FUNCTIONAL SUCCESS, NOT EXACT MATCHING:
1. Did the agent provide the requested landmark information?
2. Is the core information accurate and helpful to the user?
3. Would the user be satisfied with what they received?

DYNAMIC DATA IS EXPECTED AND CORRECT:
- Landmark search results vary based on current database state
- Different search queries may return different but valid landmarks
- Order of results may vary (this is normal for search results)
- Formatting differences are acceptable

IGNORE THESE DIFFERENCES:
- Format differences, duplicate searches, system messages
- Different result ordering or landmark selection
- Reference mismatches due to dynamic search results

MARK AS CORRECT IF:
- Agent successfully found landmarks matching the request
- User received useful, accurate landmark information
- Core functionality worked as expected (search worked, results filtered properly)

MARK AS INCORRECT ONLY IF:
- Agent completely failed to provide landmark information
- Response is totally irrelevant to the landmark search request
- Agent provided clearly wrong or nonsensical information

**Question:** {input}

**Reference Answer:** {reference}

**AI Response:** {output}

Based on the criteria above, is the AI response correct?

Answer: [correct/incorrect]

Explanation: [Provide a brief explanation focusing on functional success]
"""

# Lenient hallucination evaluation template  
LENIENT_HALLUCINATION_PROMPT_TEMPLATE = """
You are evaluating whether an AI assistant's response about landmarks contains hallucinated (fabricated) information.

DYNAMIC DATA IS EXPECTED AND FACTUAL:
- Landmark search results are pulled from a real database
- Different searches return different valid landmarks (this is correct behavior)
- Landmark details like addresses, descriptions, and activities come from actual data
- Search result variations are normal and factual

MARK AS FACTUAL IF:
- Response contains "iteration limit" or "time limit" (system issue, not hallucination)
- Agent provides plausible landmark data from search results
- Information is consistent with typical landmark search functionality
- Results differ from reference due to dynamic search (this is expected!)

ONLY MARK AS HALLUCINATED IF:
- Response contains clearly impossible landmark information
- Agent makes up fake landmark names, addresses, or details
- Response contradicts fundamental facts about landmark search
- Agent claims to have data it cannot access

REMEMBER: Different search results are EXPECTED dynamic behavior, not hallucinations!

**Question:** {input}

**Reference Answer:** {reference}

**AI Response:** {output}

Based on the criteria above, does the response contain hallucinated information?

Answer: [factual/hallucinated]

Explanation: [Focus on whether information is plausible vs clearly fabricated]
"""

# Setup the landmark search agent
agent, client = setup_landmark_agent()


2025-09-04 12:53:05,003 - INFO - ✅ Environment variables configured
2025-09-04 12:53:05,074 - INFO - A local catalog and a remote catalog have been found. Building a chained tool catalog.
2025-09-04 12:53:05,074 - INFO - A local catalog and a remote catalog have been found. Building a chained prompt catalog.
2025-09-04 12:53:05,121 - INFO - Using both a local auditor and a remote auditor.
2025-09-04 12:53:05,122 - INFO - 🔧 Setting up Priority 1 AI services for llamaindex framework...
2025-09-04 12:53:05,122 - INFO - 🔧 Endpoint: https://o1w7qdmspvermloq.ai.sandbox.nonprod-project-avengers.com
2025-09-04 12:53:05,123 - INFO - 🔧 Model: nvidia/llama-3.2-nv-embedqa-1b-v2
2025-09-04 12:53:05,123 - INFO - 🔧 API Base: https://o1w7qdmspvermloq.ai.sandbox.nonprod-project-avengers.com/v1
2025-09-04 12:53:05,123 - INFO - ✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)
2025-09-04 12:53:05,123 - INFO - 🔧 LLM Endpoint: https://o1w7qdmspvermloq.ai.sandbox.nonprod-project-avengers.com
2025-0

## Test Functions
Define test functions to demonstrate the landmark search agent functionality.


In [8]:
def run_landmark_query(query: str, agent):
    """Run a single landmark query with error handling."""
    logger.info(f"🏛️ Landmark Query: {query}")
    
    try:
        # Run the agent with LlamaIndex chat interface
        response = agent.chat(query, chat_history=[])
        result = response.response
        
        logger.info(f"🤖 AI Response: {result}")
        logger.info("✅ Query completed successfully")
        
        return result
        
    except Exception as e:
        logger.exception(f"❌ Query failed: {e}")
        return f"Error: {str(e)}"


def test_landmark_data_loading():
    """Test landmark data loading from travel-sample independently."""
    logger.info("Testing Landmark Data Loading from travel-sample")
    logger.info("=" * 50)
    
    try:
        # Import landmark data functions
        # Use inline landmark data functions (already defined in this notebook)
        # The functions get_landmark_count and get_landmark_texts are defined inline above
        
        # Test landmark count
        count = get_landmark_count()
        logger.info(f"✅ Landmark count in travel-sample.inventory.landmark: {count}")
        
        # Test landmark text generation
        texts = get_landmark_texts()
        logger.info(f"✅ Generated {len(texts)} landmark texts for embeddings")
        
        if texts:
            logger.info(f"✅ First landmark text sample: {texts[0][:200]}...")
        
        logger.info("✅ Data loading test completed successfully")
        
    except Exception as e:
        logger.exception(f"❌ Data loading test failed: {e}")


# Test landmark data loading
test_landmark_data_loading()


2025-09-04 13:00:14,826 - INFO - Testing Landmark Data Loading from travel-sample
2025-09-04 13:00:14,828 - INFO - ==================================================
2025-09-04 13:00:17,554 - INFO - ✅ Landmark count in travel-sample.inventory.landmark: 4495
2025-09-04 13:00:19,742 - INFO - Loading landmark data from travel-sample.inventory.landmark...
2025-09-04 13:00:19,743 - INFO - Processing landmark documents...
Loading landmarks: 100%|██████████| 4495/4495 [00:00<00:00, 4742992.83landmarks/s]
2025-09-04 13:00:24,967 - INFO - Loaded 4495 landmarks from travel-sample.inventory.landmark
2025-09-04 13:00:24,969 - INFO - Generating landmark text embeddings...
Processing landmarks: 100%|██████████| 4495/4495 [00:00<00:00, 212114.76landmarks/s]
2025-09-04 13:00:24,992 - INFO - Generated 4495 landmark text embeddings
2025-09-04 13:00:24,995 - INFO - ✅ Generated 4495 landmark texts for embeddings
2025-09-04 13:00:24,996 - INFO - ✅ First landmark text sample: San Francisco/Haight (&quot;Hip

## Test 1: Landmarks in Tokyo

Search for landmarks and attractions in Tokyo, Japan.


In [9]:
result1 = run_landmark_query("Find me landmarks in Tokyo", agent)


2025-09-04 13:00:25,004 - INFO - 🏛️ Landmark Query: Find me landmarks in Tokyo


> Running step 84e9b29f-a623-4e14-ba60-e9e3feedd07c. Step input: Find me landmarks in Tokyo
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_landmarks
Action Input: {'query': 'Tokyo', 'limit': 5}


2025-09-04 13:00:35,284 - INFO - Search query: 'Tokyo' found 5 results


Observation: Found 5 landmarks matching 'Tokyo':

1. **Hanawa**
   📍 Location: Paris, France
   🗺️ State: Île-de-France.
   🎯 Activity: Eat.
   🏠 Address: 26, rue Bayard.
   📞 Phone: +33 1 56 62 70 70.
   📝 Description: Great sushi in a nice atmosphere, extensive menu..

2. **Sapporo**
   📍 Location: Liverpool, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 134 Duke Street, East Village.
   📞 Phone: +44 151 709-4002.
   🌐 Website: http://www.sapporo.co.uk/restaurant-liverpool/.
   📝 Description: Japanese cuisine.

3. **Mikawaya**
   📍 Location: Los Angeles, United States
   🗺️ State: California.
   🎯 Activity: Eat.
   🏠 Address: 118 Japanese Village Plaza Mall.
   📞 Phone: +1 213 624-1681.
   🌐 Website: http://www.mikawayausa.com.
   🕒 Hours: M,Sa 10AM-7PM, Tu-Th 10AM-10PM, F 9AM-11PM, Su closed.
   📝 Description: Their motto says it best: &quot;The finest name in Japanese pastries since 1910.&quot;.

4. **Tru**
   📍 Location: Camberley, United Kingdom
   🎯 Activity: Drink.
   🏠 Addr

2025-09-04 13:00:45,569 - INFO - Search query: 'Tokyo landmarks' found 5 results


...

Observation: Found 5 landmarks matching 'Tokyo, Japan landmarks':

1. **Hanawa**
   📍 Location: Paris, France
   🗺️ State: Île-de-France.
   🎯 Activity: Eat.
   🏠 Address: 26, rue Bayard.
   📞 Phone: +33 1 56 62 70 70.
   📝 Description: Great sushi in a nice atmosphere, extensive menu..

2. **Sapporo**
   📍 Location: Liverpool, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 134 Duke Street, East Village.
   📞 Phone: +44 151 709-4002.
   🌐 Website: http://www.sapporo.co.uk/restaurant-liverpool/.
   📝 Description: Japanese cuisine.

3. **Sapporo-Ya Japanese Restaurant**
   📍 Location: San Francisco, United States
   🗺️ State: California.
   🎯 Activity: Eat.
   🏠 Address: 1581 Webster Street.
   📞 Phone: +1 415 563-7400.
   🕒 Hours: M-Sa 11AM-11PM, Su 11AM-10:30PM.
   📝 Description: Excellent Japanese noodles served dozens of ways. Open very late, and a good place to warm up on a cold foggy day..

4. **Webster Bridge**
   📍 Location: San Francisco, United States
   🗺️ State: California.

2025-09-04 13:01:17,413 - INFO - Search query: 'Tokyo, Japan landmarks' found 10 results


Observation: Found 10 landmarks matching 'Tokyo, Japan landmarks':

1. **Hanawa**
   📍 Location: Paris, France
   🗺️ State: Île-de-France.
   🎯 Activity: Eat.
   🏠 Address: 26, rue Bayard.
   📞 Phone: +33 1 56 62 70 70.
   📝 Description: Great sushi in a nice atmosphere, extensive menu..

2. **Sapporo**
   📍 Location: Liverpool, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 134 Duke Street, East Village.
   📞 Phone: +44 151 709-4002.
   🌐 Website: http://www.sapporo.co.uk/restaurant-liverpool/.
   📝 Description: Japanese cuisine.

3. **Sapporo-Ya Japanese Restaurant**
   📍 Location: San Francisco, United States
   🗺️ State: California.
   🎯 Activity: Eat.
   🏠 Address: 1581 Webster Street.
   📞 Phone: +1 415 563-7400.
   🕒 Hours: M-Sa 11AM-11PM, Su 11AM-10:30PM.
   📝 Description: Excellent Japanese noodles served dozens of ways. Open very late, and a good place to warm up on a cold foggy day..

4. **Webster Bridge**
   📍 Location: San Francisco, United States
   🗺️ State: California

2025-09-04 13:01:19,908 - ERROR - ❌ Query failed: Reached max iterations.
Traceback (most recent call last):
  File "/tmp/ipykernel_97189/2404050830.py", line 7, in run_landmark_query
    response = agent.chat(query, chat_history=[])
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaustav/.cache/pypoetry/virtualenvs/landmark-search-agent-2081kaWT-py3.12/lib/python3.12/site-packages/llama_index_instrumentation/dispatcher.py", line 317, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaustav/.cache/pypoetry/virtualenvs/landmark-search-agent-2081kaWT-py3.12/lib/python3.12/site-packages/llama_index/core/callbacks/utils.py", line 42, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kaustav/.cache/pypoetry/virtualenvs/landmark-search-agent-2081kaWT-py3.12/lib/python3.12/site-packages/llama_index/core/agent/runner/base.py", line 708, in chat
    chat_response = self._chat(
    

Thought: The search results still do not seem to be related to Tokyo landmarks. I need to use the tool again to get the correct results. I will try to specify the location of the landmarks as Tokyo, Japan, and also try to get more results from a different database.
Action: search_landmarks
Action Input: {'query': 'Tokyo landmarks', 'limit': 10, 'database': 'tokyo-sample'}
Observation: Error: search_landmarks() got an unexpected keyword argument 'database'


## Test 2: Museums in London

Search for museums and cultural attractions in London, UK.


In [10]:
result2 = run_landmark_query("Show me museums in London", agent)


2025-09-04 13:01:19,926 - INFO - 🏛️ Landmark Query: Show me museums in London


> Running step c428d54a-9b12-481d-92a3-1abf55b4ee8c. Step input: Show me museums in London
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_landmarks
Action Input: {'query': 'museums in London', 'limit': 5}


2025-09-04 13:01:29,156 - INFO - Search query: 'museums in London' found 5 results


Observation: Found 4 landmarks matching 'museums in London':

1. **London Transport Museum**
   📍 Location: London, United Kingdom
   🎯 Activity: See.
   🌐 Website: http://www.ltmuseum.co.uk.
   📝 Description: [[London]] (in [[London/Covent Garden|Covent Garden]]).

2. **Clockmaker's Museum**
   📍 Location: London, United Kingdom
   🎯 Activity: See.
   🏠 Address: Guildhall Library, Aldermanbury EC2P 2EJ.
   📞 Phone: +44 20 7332-1868. Email: printedbooks.guildhall@corpoflondon.gov.uk.
   🌐 Website: http://www.clockmakers.org/.
   🕒 Hours: M–Sa 09:30–17:00.
   💰 Price: Free.
   📝 Description: Charts the history of clockmaking and houses a priceless collection of old timepieces..

4. **575 Wandsworth Road**
   📍 Location: London, United Kingdom
   🎯 Activity: See.
   🏠 Address: 575 Wandsworth Road, Lambeth, London, London, SW8 3JD.
   📞 Phone: +44 20 7720-9459. Email: 575wandsworthroad@nationaltrust.org.uk.
   🌐 Website: http://www.nationaltrust.org.uk/575-wandsworth-road/.
   📝 Descripti

2025-09-04 13:01:31,041 - INFO - 🤖 AI Response: There are several museums in London that you might be interested in visiting. Some of the options include the London Transport Museum, the Clockmaker's Museum, and 575 Wandsworth Road. Each of these museums offers a unique perspective on history and culture, and they are all located in different parts of the city.
2025-09-04 13:01:31,042 - INFO - ✅ Query completed successfully


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: There are several museums in London that you might be interested in visiting. Some of the options include the London Transport Museum, the Clockmaker's Museum, and 575 Wandsworth Road. Each of these museums offers a unique perspective on history and culture, and they are all located in different parts of the city.


## Test 3: Parks in Paris

Search for parks and outdoor spaces in Paris, France.


In [11]:
result3 = run_landmark_query("What parks can I visit in Paris?", agent)


2025-09-04 13:01:31,048 - INFO - 🏛️ Landmark Query: What parks can I visit in Paris?


> Running step fa0c2fcb-5608-4ba2-9c7d-07f7064db0a8. Step input: What parks can I visit in Paris?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_landmarks
Action Input: {'query': 'parks in Paris', 'limit': 5}


2025-09-04 13:01:39,938 - INFO - Search query: 'parks in Paris' found 5 results


Observation: Found 5 landmarks matching 'parks in Paris':

1. **Parc André Malraux**
   📍 Location: Nanterre, France
   🗺️ State: Île-de-France.
   🎯 Activity: See.
   📝 Description: The expansive, calm park around a lake extends westwards behind the Tours Aillaud, and is to La Defense quite what Central Park is to downtown Manhattan. You can take spectaular pictures of the La Defense skyline juxtaposted against the park's greenery from there..

2. **Madeleine**
   📍 Location: Paris, France
   🗺️ State: Île-de-France.
   🎯 Activity: Listing.
   📝 Description: (lines 8, 12 and 14).

3. **Parc André Citroën**
   📍 Location: Paris, France
   🗺️ State: Île-de-France. Image: https://en.wikivoyage.org/wiki/File:Jardin André Citroën (PARIS,FR75) (3830986538).jpg.
   🎯 Activity: See.
   🏠 Address: 2, rue Cauchy.
   🌐 Website: http://equipement.paris.fr/parc-andre-citroen-1791.
   🕒 Hours: 08:00-17:45.
   📝 Description: The large park occupies the 14 ha formerly occupied by a Citroën factory. S

2025-09-04 13:01:41,646 - INFO - 🤖 AI Response: You can visit the following parks in Paris: Parc André Malraux, Parc André Citroën, and Musée en Herbe.
2025-09-04 13:01:41,647 - INFO - ✅ Query completed successfully


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: You can visit the following parks in Paris: Parc André Malraux, Parc André Citroën, and Musée en Herbe.


## Comprehensive Phoenix Evaluation System

Complete Phoenix evaluation system from evals/eval_arize.py - inline for self-contained operation.


In [12]:
# Comprehensive Phoenix evaluation system from evals/eval_arize.py
import nest_asyncio
import socket
import subprocess
import warnings
from dataclasses import dataclass
from datetime import datetime
from typing import Any, Dict, List, Optional

import pandas as pd

# Apply nest_asyncio to handle nested event loops in Jupyter/LlamaIndex
nest_asyncio.apply()

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

@dataclass
class EvaluationConfig:
    """Configuration for the evaluation system."""

    # Arize Configuration
    arize_space_id: str = os.getenv("ARIZE_SPACE_ID", "default-space")
    arize_api_key: str = os.getenv("ARIZE_API_KEY", "")
    project_name: str = "landmark-search-agent-evaluation"

    # Phoenix Configuration
    phoenix_base_port: int = 6006
    phoenix_grpc_base_port: int = 4317
    phoenix_max_port_attempts: int = 5

    # Evaluation Configuration
    evaluator_model: str = "gpt-4o"
    max_queries: int = 10
    evaluation_timeout: int = 300


class PhoenixManager:
    """Manages Phoenix server lifecycle."""

    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.session = None
        self.active_port = None

    def _is_port_in_use(self, port: int) -> bool:
        """Check if a port is in use."""
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            return s.connect_ex(("localhost", port)) == 0

    def _kill_existing_phoenix_processes(self) -> None:
        """Kill any existing Phoenix processes."""
        try:
            subprocess.run(["pkill", "-f", "phoenix"], check=False, capture_output=True)
            time.sleep(2)  # Wait for processes to terminate
        except Exception as e:
            logger.debug(f"Error killing Phoenix processes: {e}")

    def _find_available_port(self) -> tuple[int, int]:
        """Find available ports for Phoenix."""
        phoenix_port = self.config.phoenix_base_port
        grpc_port = self.config.phoenix_grpc_base_port

        for _ in range(self.config.phoenix_max_port_attempts):
            if not self._is_port_in_use(phoenix_port):
                return phoenix_port, grpc_port
            phoenix_port += 1
            grpc_port += 1

        raise RuntimeError(
            f"Could not find available ports after {self.config.phoenix_max_port_attempts} attempts"
        )

    def start_phoenix(self) -> bool:
        """Start Phoenix server and return success status."""
        try:
            import phoenix as px
            from phoenix.otel import register
            from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
            
            logger.info("🔧 Setting up Phoenix observability...")

            # Clean up existing processes
            self._kill_existing_phoenix_processes()

            # Find available ports
            phoenix_port, grpc_port = self._find_available_port()

            # Set environment variables
            os.environ["PHOENIX_PORT"] = str(phoenix_port)
            os.environ["PHOENIX_GRPC_PORT"] = str(grpc_port)

            # Start Phoenix session
            self.session = px.launch_app()
            self.active_port = phoenix_port

            if self.session:
                logger.info(f"🌐 Phoenix UI: {self.session.url}")

            # Register Phoenix OTEL for LlamaIndex
            register(
                project_name=self.config.project_name,
                endpoint=f"http://localhost:{self.active_port}/v1/traces",
            )

            # Instrument LlamaIndex specifically
            LlamaIndexInstrumentor().instrument()

            logger.info("✅ Phoenix setup completed successfully")
            return True

        except Exception as e:
            logger.exception(f"❌ Phoenix setup failed: {e}")
            return False

    def cleanup(self) -> None:
        """Clean up Phoenix resources."""
        try:
            if self.session:
                # Phoenix session cleanup happens automatically
                pass
            logger.info("🔒 Phoenix cleanup completed")
        except Exception as e:
            logger.warning(f"⚠️ Error during Phoenix cleanup: {e}")


class LandmarkSearchEvaluator:
    """LlamaIndex-specific evaluator for the landmark search agent."""

    def __init__(self, config: Optional[EvaluationConfig] = None):
        """Initialize the evaluator with configuration."""
        self.config = config or EvaluationConfig()
        self.phoenix_manager = PhoenixManager(self.config)

        # Agent components
        self.agent = None
        self.client = None

        # Phoenix evaluators
        self.evaluator_llm = None

        # Add option to bypass Phoenix for debugging
        try:
            import phoenix as px
            if not os.getenv("SKIP_PHOENIX", "false").lower() == "true":
                self._setup_phoenix_evaluators()
            elif os.getenv("SKIP_PHOENIX", "false").lower() == "true":
                logger.info("🔧 Phoenix setup skipped due to SKIP_PHOENIX=true")
        except ImportError:
            logger.warning("Phoenix not available - skipping Phoenix setup")

    def _setup_phoenix_evaluators(self) -> None:
        """Setup Phoenix evaluators for LLM-based evaluation."""
        try:
            from phoenix.evals import OpenAIModel
            
            self.evaluator_llm = OpenAIModel(model=self.config.evaluator_model)
            logger.info("✅ Phoenix evaluators initialized")

            # Start Phoenix
            if self.phoenix_manager.start_phoenix():
                logger.info("✅ Phoenix instrumentation enabled for LlamaIndex")

        except Exception as e:
            logger.warning(f"⚠️ Phoenix evaluators setup failed: {e}")
            self.evaluator_llm = None

    def setup_agent(self) -> bool:
        """Setup landmark search agent using the setup function."""
        try:
            logger.info("🔧 Setting up landmark search agent...")

            self.agent, self.client = setup_landmark_agent()

            logger.info("✅ Landmark search agent setup completed successfully")
            return True

        except Exception as e:
            logger.exception(f"❌ Error setting up landmark search agent: {e}")
            return False

    def _extract_response_content(self, result: Any) -> str:
        """Extract clean response content from LlamaIndex agent result."""
        try:
            # Prefer explicit response field
            if hasattr(result, "response"):
                response_content = str(result.response).strip()
                if response_content and not response_content.lower().startswith("error:"):
                    return response_content

            # Some LlamaIndex results may carry a .message or .output
            for attr in ("message", "output", "final_response"):
                if hasattr(result, attr):
                    text = str(getattr(result, attr)).strip()
                    if text:
                        return text

            # Last resort fallback
            text = str(result).strip()
            return text if text else ""
                
        except Exception as e:
            logger.warning(f"Error extracting response content: {e}")
            return f"Error extracting response: {e}"

    def run_single_evaluation(self, query: str) -> Dict[str, Any]:
        """Run evaluation for a single query using LlamaIndex agent."""
        if not self.agent:
            raise RuntimeError("Agent not initialized. Call setup_agent() first.")

        logger.info(f"🔍 Evaluating query: {query}")

        start_time = time.time()

        try:
            # Use LlamaIndex .chat() method
            result = self.agent.chat(query, chat_history=[])

            # Extract response content
            response = self._extract_response_content(result)

            # Create evaluation result
            evaluation_result = {
                "query": query,
                "response": response,
                "execution_time": time.time() - start_time,
                "success": True,
                "sources": [],
                "num_sources": 0,
            }

            logger.info(f"✅ Query completed in {evaluation_result['execution_time']:.2f}s")

            return evaluation_result

        except Exception as e:
            logger.exception(f"❌ Query failed: {e}")
            return {
                "query": query,
                "response": f"Error: {str(e)}",
                "execution_time": time.time() - start_time,
                "success": False,
                "error": str(e),
                "sources": [],
                "num_sources": 0,
            }

    def run_phoenix_evaluations(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Run Phoenix evaluations on the results."""
        if not self.evaluator_llm:
            logger.warning("⚠️ Phoenix evaluators not available - skipping LLM evaluations")
            return results_df

        logger.info(f"🧠 Running Phoenix evaluations on {len(results_df)} responses...")

        try:
            from phoenix.evals import (
                RAG_RELEVANCY_PROMPT_TEMPLATE,
                RAG_RELEVANCY_PROMPT_RAILS_MAP,
                TOXICITY_PROMPT_TEMPLATE,
                TOXICITY_PROMPT_RAILS_MAP,
                llm_classify,
            )
            
            # Prepare evaluation data
            evaluation_data = []
            for _, row in results_df.iterrows():
                query = row["query"]
                response = row["response"]

                # Get reference answer for this query
                reference = self._get_reference_answer(str(query))

                evaluation_data.append(
                    {
                        "input": query,
                        "output": response,
                        "reference": reference,
                        "context": "Landmark search results",
                        "text": response,  # For toxicity evaluation
                    }
                )

            eval_df = pd.DataFrame(evaluation_data)

            # Run individual Phoenix evaluations
            self._run_individual_phoenix_evaluations(eval_df, results_df)

            logger.info("✅ Phoenix evaluations completed")

        except Exception as e:
            logger.exception(f"❌ Error running Phoenix evaluations: {e}")
            # Add error indicators
            for eval_type in ["relevance", "qa_correctness", "hallucination", "toxicity"]:
                results_df[eval_type] = "error"
                results_df[f"{eval_type}_explanation"] = f"Error: {e}"

        return results_df

    def _get_reference_answer(self, query: str) -> str:
        """Get reference answer for evaluation."""
        try:
            reference_answer = get_reference_answer(query)

            if reference_answer.startswith("No reference answer available"):
                # Create a basic reference based on query
                if "museum" in query.lower() or "gallery" in query.lower():
                    return "Should provide information about museums and galleries with accurate names, addresses, and descriptions."
                elif "restaurant" in query.lower() or "food" in query.lower():
                    return "Should provide information about restaurants and food establishments."
                else:
                    return "Should provide relevant and accurate landmark information."

            return reference_answer

        except Exception as e:
            logger.warning(f"Could not get reference answer for '{query}': {e}")
            return "Should provide relevant and accurate landmark information."

    def run_evaluation(self, queries: List[str]) -> pd.DataFrame:
        """Run complete evaluation pipeline."""
        if not self.setup_agent():
            raise RuntimeError("Failed to setup agent")

        # Limit queries if specified
        if len(queries) > self.config.max_queries:
            queries = queries[: self.config.max_queries]
            logger.info(f"Limited to {self.config.max_queries} queries for evaluation")

        logger.info(
            f"🚀 Starting LlamaIndex landmark search evaluation with {len(queries)} queries"
        )

        # Run queries
        results = []
        for i, query in enumerate(queries, 1):
            logger.info(f"\n📋 Query {i}/{len(queries)}")
            result = self.run_single_evaluation(query)
            results.append(result)

        # Create results DataFrame
        results_df = pd.DataFrame(results)

        # Run Phoenix evaluations
        results_df = self.run_phoenix_evaluations(results_df)

        # Log summary
        self._log_evaluation_summary(results_df)

        return results_df

    def _log_evaluation_summary(self, results_df: pd.DataFrame) -> None:
        """Log evaluation summary."""
        logger.info("\n📊 Evaluation Summary:")
        logger.info(f"  Total queries: {len(results_df)}")
        logger.info(f"  Successful executions: {results_df['success'].sum()}")
        logger.info(f"  Failed executions: {(~results_df['success']).sum()}")
        logger.info(f"  Average execution time: {results_df['execution_time'].mean():.2f}s")

    def cleanup(self) -> None:
        """Clean up all resources."""
        self.phoenix_manager.cleanup()


def get_default_queries() -> List[str]:
    """Get default test queries for evaluation."""
    return get_queries_for_evaluation(limit=10)


def run_comprehensive_evaluation() -> pd.DataFrame:
    """Run comprehensive evaluation with all Phoenix evaluators."""
    evaluator = LandmarkSearchEvaluator()
    try:
        queries = get_default_queries()
        results = evaluator.run_evaluation(queries)
        logger.info("\n✅ Comprehensive landmark search evaluation complete!")
        return results
    finally:
        evaluator.cleanup()


## Demo and Test Functions

Interactive demo and test functions from main.py - inline for self-contained operation.


In [13]:
# Demo and test functions from main.py

def run_interactive_demo():
    """Run an interactive landmark search demo."""
    logger.info("Landmark Search Agent - Interactive Demo")
    logger.info("=" * 50)

    try:
        agent, client = setup_landmark_agent()

        # Interactive landmark search loop
        logger.info("Available commands:")
        logger.info("- Enter landmark search queries (e.g., 'Find landmarks in Paris')")
        logger.info("- 'quit' - Exit the demo")
        logger.info("Try asking: 'Find me landmarks in Tokyo' or 'Show me museums in London'")
        logger.info("─" * 40)

        while True:
            query = input("🔍 Enter landmark search query (or 'quit' to exit): ").strip()

            if query.lower() in ["quit", "exit", "q"]:
                logger.info("Thanks for using Landmark Search Agent!")
                break

            if not query:
                logger.warning("Please enter a search query")
                continue

            try:
                response = agent.chat(query, chat_history=[])
                result = response.response

                logger.info(f"\n🏛️ Agent Response:\n{result}\n")
                logger.info("─" * 40)

            except Exception as e:
                logger.error(f"Error processing query: {e}")
                logger.error(f"❌ Error: {e}")
                logger.info("─" * 40)

    except KeyboardInterrupt:
        logger.info("Demo interrupted by user")
    except Exception as e:
        logger.exception(f"Demo error: {e}")
    finally:
        logger.info("Demo completed")


def run_test():
    """Run comprehensive test of landmark search agent with queries from queries.py."""
    logger.info("Landmark Search Agent - Comprehensive Test Suite")
    logger.info("=" * 55)

    try:
        agent, client = setup_landmark_agent()

        # Test scenarios covering different types of landmark searches
        test_queries = get_queries_for_evaluation()

        logger.info(f"Running {len(test_queries)} test queries...")

        for i, query in enumerate(test_queries, 1):
            logger.info(f"\n🔍 Test {i}: {query}")
            try:
                response = agent.chat(query, chat_history=[])
                result = response.response

                # Display the response
                logger.info(f"🤖 AI Response: {result}")
                logger.info(f"✅ Test {i} completed successfully")

            except Exception as e:
                logger.exception(f"❌ Test {i} failed: {e}")

            logger.info("-" * 50)

        logger.info("All tests completed!")

    except Exception as e:
        logger.exception(f"Test error: {e}")


def run_landmark_data_test():
    """Test landmark data loading and querying functions."""
    logger.info("Landmark Data Test Suite")
    logger.info("=" * 30)
    
    try:
        # Test landmark count
        count = get_landmark_count()
        logger.info(f"✅ Landmark count in travel-sample.inventory.landmark: {count}")
        
        # Test data availability - basic validation
        logger.info("\n✅ Basic landmark data validation completed successfully!")
        
    except Exception as e:
        logger.exception(f"❌ Landmark data test error: {e}")


# Quick demo functions for notebook use
def run_demo_queries():
    """Run a few demo queries for quick testing."""
    demo_queries = [
        "Find museums and galleries in Glasgow",
        "Tell me about Monet's House",
        "What attractions can I see in Glasgow?"
    ]
    
    agent, client = setup_landmark_agent()
    
    for i, query in enumerate(demo_queries, 1):
        logger.info(f"\n🔍 Demo Query {i}: {query}")
        try:
            response = agent.chat(query, chat_history=[])
            logger.info(f"🏛️ Response: {response.response}")
        except Exception as e:
            logger.error(f"❌ Error: {e}")
        logger.info("-" * 50)


# Run comprehensive evaluation (uses the Phoenix evaluator above)
def run_evaluation():
    """Run the comprehensive Phoenix evaluation."""
    return run_comprehensive_evaluation()


## Arize Phoenix Evaluation

This section demonstrates how to evaluate the landmark search agent using Arize Phoenix observability platform. The evaluation includes:

- **Relevance Scoring**: Using Phoenix RelevanceEvaluator to score how relevant responses are to queries
- **QA Scoring**: Using Phoenix QAEvaluator to score answer quality
- **Hallucination Detection**: Using Phoenix HallucinationEvaluator to detect fabricated information  
- **Toxicity Detection**: Using Phoenix ToxicityEvaluator to detect harmful content
- **Phoenix UI**: Real-time observability dashboard at `http://localhost:6006/`

We'll run landmark search queries and evaluate the responses for quality and safety using LlamaIndex instrumentation.


In [14]:
# Import Phoenix evaluation components
try:
    import phoenix as px
    from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
    from phoenix.evals import (
        HALLUCINATION_PROMPT_RAILS_MAP,
        HALLUCINATION_PROMPT_TEMPLATE,
        QA_PROMPT_RAILS_MAP,
        QA_PROMPT_TEMPLATE,
        RAG_RELEVANCY_PROMPT_RAILS_MAP,
        RAG_RELEVANCY_PROMPT_TEMPLATE,
        TOXICITY_PROMPT_RAILS_MAP,
        TOXICITY_PROMPT_TEMPLATE,
        OpenAIModel,
        llm_classify,
    )
    from phoenix.otel import register
    import pandas as pd
    
    ARIZE_AVAILABLE = True
    logger.info("✅ Arize Phoenix evaluation components available")
except ImportError as e:
    logger.warning(f"Arize dependencies not available: {e}")
    logger.warning("Skipping evaluation section...")
    ARIZE_AVAILABLE = False

if ARIZE_AVAILABLE:
    # Start Phoenix session for observability
    try:
        px.launch_app(port=6006)
        logger.info("🚀 Phoenix UI available at http://localhost:6006/")
        
        # Register LlamaIndex instrumentation
        tracer_provider = register(
            project_name="landmark-search-agent-evaluation",
            endpoint="http://localhost:6006/v1/traces"
        )
        
        # Instrument LlamaIndex
        LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)
        logger.info("✅ LlamaIndex instrumentation enabled")
        
    except Exception as e:
        logger.warning(f"Could not start Phoenix UI: {e}")

    # Demo queries for evaluation
    landmark_demo_queries = [
        "Find museums and galleries in Glasgow",
        "Show me restaurants serving Asian cuisine", 
        "What attractions can I see in Glasgow?",
        "Tell me about Monet's House",
        "Find places to eat in Gillingham"
    ]
    
    # Run demo queries and collect responses for evaluation
    landmark_demo_results = []
    
    for i, query in enumerate(landmark_demo_queries, 1):
        try:
            logger.info(f"🔍 Running evaluation query {i}: {query}")
            
            # Run the agent with LlamaIndex
            response = agent.chat(query, chat_history=[])
            output = response.response
    
            landmark_demo_results.append({
                "query": query,
                "response": output,
                "query_type": f"landmark_demo_{i}",
                "success": True
            })
            
            logger.info(f"✅ Query {i} completed successfully")
    
        except Exception as e:
            logger.exception(f"❌ Query {i} failed: {e}")
            landmark_demo_results.append({
                "query": query,
                "response": f"Error: {e!s}",
                "query_type": f"landmark_demo_{i}",
                "success": False
            })
    
    # Convert to DataFrame for evaluation
    landmark_results_df = pd.DataFrame(landmark_demo_results)
    logger.info(f"📊 Collected {len(landmark_results_df)} responses for evaluation")
    
    # Display results summary
    for _, row in landmark_results_df.iterrows():
        logger.info(f"Query: {row['query']}")
        logger.info(f"Response: {row['response'][:200]}...")
        logger.info(f"Success: {row['success']}")
        logger.info("-" * 50)
    
    logger.info("💡 Visit Phoenix UI at http://localhost:6006/ to see detailed traces and evaluations")
    logger.info("💡 Use the evaluation script at evals/eval_arize.py for comprehensive evaluation")

else:
    logger.info("Arize evaluation not available - install phoenix-evals to enable evaluation")


2025-09-04 13:01:41,771 - INFO - 📋 Ensuring phoenix working directory: /home/kaustav/.phoenix
2025-09-04 13:01:41,807 - INFO - Dataset: phoenix_inferences_70c15d2e-2e52-455b-b6db-149b436b6c95 initialized
2025-09-04 13:01:43,242 - INFO - ✅ Arize Phoenix evaluation components available
2025-09-04 13:01:43,244 - INFO - 📋 Ensuring phoenix working directory: /home/kaustav/.phoenix
2025-09-04 13:01:43,302 - INFO - Context impl SQLiteImpl.
2025-09-04 13:01:43,304 - INFO - Will assume transactional DDL.
2025-09-04 13:01:43,342 - INFO - Running upgrade  -> cf03bd6bae1d, init


❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


2025-09-04 13:01:43,917 - INFO - Running upgrade cf03bd6bae1d -> 10460e46d750, datasets
2025-09-04 13:01:44,302 - INFO - Running upgrade 10460e46d750 -> 3be8647b87d8, add token columns to spans table
2025-09-04 13:01:44,305 - INFO - Running upgrade 3be8647b87d8 -> cd164e83824f, users and tokens
2025-09-04 13:01:44,313 - INFO - Running upgrade cd164e83824f -> 4ded9e43755f, create project_session table
2025-09-04 13:01:44,324 - INFO - Running upgrade 4ded9e43755f -> bc8fea3c2bc8, Add prompt tables
2025-09-04 13:01:44,330 - INFO - Running upgrade bc8fea3c2bc8 -> 2f9d1a65945f, Annotation config migrations
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)
2025-09-04 13:01:44,424 - INFO - Running upgrade 2f9d1a65945f -> bb8139330879, create project tr

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: landmark-search-agent-evaluation
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

> Running step 17677ca9-e3a6-4ef0-abc2-e5fcf43d8bc9. Step input: Find museums and galleries in Glasgow
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_landmarks
Action Input:

2025-09-04 13:01:53,807 - INFO - Search query: 'museums and galleries in Glasgow' found 5 results


...

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The landmark you are referring to is likely the house of Claude Monet, a famous French painter, located in Giverny, France. The house is now a museum and is known for its beautiful gardens, which were a major inspiration for Monet's paintings. The gardens feature a water garden with a Japanese bridge, weeping willows, and water lilies, which are now iconic symbols of Monet's work. The house itself is a unique and interesting example of Oriental-influenced architecture, and it includes Monet's collection of Japanese prints. Visitors can take a guided tour of the house and gardens, and there is also a gift store on site where you can purchase souvenirs.
> Running step e849f9b0-0194-4864-9e87-0740511abe5f. Step input: Find places to eat in Gillingham
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_landmarks
Action Inp

2025-09-04 13:02:40,563 - INFO - Search query: 'places to eat in Gillingham' found 5 results


Observation: Found 5 landmarks matching 'places to eat in Gillingham':

1. **Beijing Inn**
   📍 Location: Gillingham, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 3 King Street, ME6 1EY.
   🌐 Website: http://beijinginn.co.uk/div/.
   📝 Description: Chinese restaurant just off the High Street..

2. **Ossie's Fish and Chips**
   📍 Location: Gillingham, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 75 Richmond Road, ME7 1LS.
   📞 Phone: +44 1634 582000.
   📝 Description: Best fish and chips in the area..

3. **The Bridge Brasserie**
   📍 Location: Chippenham, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 29 New Road, Chippenham SN15 1HZ.
   📞 Phone: +44 1249 444552.
   🌐 Website: http://thebridgebrasserie.co.uk.
   📝 Description: Casual fine dining and great cocktails, a seasonal à la carte menu, and free WiFi.

4. **Saffron**
   📍 Location: Bedford, United Kingdom
   🎯 Activity: Eat.
   🏠 Address: 64 Tavistock St, MK40 2RG.
   📞 Phone: +44 1234 325655.
   📝 Description: Does a 

2025-09-04 13:02:43,265 - INFO - ✅ Query 5 completed successfully
2025-09-04 13:02:43,266 - INFO - 📊 Collected 5 responses for evaluation
2025-09-04 13:02:43,267 - INFO - Query: Find museums and galleries in Glasgow
2025-09-04 13:02:43,268 - INFO - Response: Based on the search results, there are four museums and galleries in Glasgow that you might be interested in visiting: The Tron Theatre, Kelvingrove Art Gallery and Museum, Riverside Museum, and Centr...
2025-09-04 13:02:43,268 - INFO - Success: True
2025-09-04 13:02:43,269 - INFO - --------------------------------------------------
2025-09-04 13:02:43,269 - INFO - Query: Show me restaurants serving Asian cuisine
2025-09-04 13:02:43,269 - INFO - Response: Here are some restaurants serving Asian cuisine: New Canton, World Curry, So Asia, Old Thai House, and La Perle d'Asie....
2025-09-04 13:02:43,269 - INFO - Success: True
2025-09-04 13:02:43,269 - INFO - --------------------------------------------------
2025-09-04 13:02:43,270 - I

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Based on the search results, here are 5 places to eat in Gillingham: Beijing Inn, Ossie's Fish and Chips, The Bridge Brasserie (note: this is actually in Chippenham, not Gillingham), Saffron (note: this is actually in Bedford, not Gillingham), and Ashbourne Fishbar (note: this is actually in Ashbourne, not Gillingham). It seems that the search results included some incorrect locations. If you're looking for places to eat in Gillingham, I would recommend checking online review sites such as TripAdvisor or Yelp for more accurate and up-to-date information.


In [15]:
if ARIZE_AVAILABLE and len(landmark_demo_results) > 0:
    logger.info("🔍 Running comprehensive Phoenix evaluations...")
    
    # Setup evaluator LLM (using OpenAI for consistency)
    evaluator_llm = OpenAIModel(model="gpt-4o", temperature=0.1)
    
    # Reference answers copied from data/queries.py (proper copy-paste as requested)
    LANDMARK_REFERENCE_ANSWERS = [
        # Query 1: Glasgow museums and galleries
        """Glasgow has several museums and galleries including the Gallery of Modern Art (Glasgow) located at Royal Exchange Square with a terrific collection of recent paintings and sculptures, the Kelvingrove Art Gallery and Museum on Argyle Street with one of the finest civic collections in Europe including works by Van Gogh, Monet and Rembrandt, the Hunterian Museum and Art Gallery at University of Glasgow with a world famous Whistler collection, and the Riverside Museum at 100 Pointhouse Place with an excellent collection of vehicles and transport history. All offer free admission except for special exhibitions.""",
        # Query 2: Asian cuisine restaurants
        """There are several Asian restaurants available including Shangri-la Chinese Restaurant in Birmingham at 51 Station Street offering good quality Chinese food with spring rolls and sizzling steak, Taiwan Restaurant in San Francisco famous for their dumplings, Hong Kong Seafood Restaurant in San Francisco for sit-down dim sum, Cheung Hing Chinese Restaurant in San Francisco for Cantonese BBQ and roast duck, Vietnam Restaurant in San Francisco for Vietnamese dishes including crab soup and pork sandwich, and various other Chinese and Asian establishments across different locations.""",
        # Query 3: Glasgow attractions
        """Glasgow attractions include Glasgow Green (founded by Royal grant in 1450) with Nelson's Memorial and the Doulton Fountain, Glasgow University (founded 1451) with neo-Gothic architecture and commanding views, Glasgow Cathedral with fine Gothic architecture from medieval times, the City Chambers in George Square built in 1888 in Italian Renaissance style with guided tours available, Glasgow Central Station with its grand interior, and Kelvingrove Park which is popular with students and contains the Art Gallery and Museum.""",
        # Query 4: Monet's House
        """Monet's House is located in Giverny, France at 84 rue Claude Monet. The house is quietly eccentric and highly interesting in an Orient-influenced style, featuring Monet's collection of Japanese prints. The main attraction is the gardens around the house, including the water garden with the Japanese bridge, weeping willows and waterlilies which are now iconic. It's open April-October, Monday-Sunday 9:30-18:00, with admission €9 for adults, €5 for students, €4 for disabled visitors, and free for under-7s. E-tickets can be purchased online and wheelchair access is available.""",
        # Query 5: Gillingham restaurants
        """Gillingham has various dining options including Beijing Inn (Chinese restaurant at 3 King Street), Spice Court (Indian restaurant at 56-58 Balmoral Road opposite the railway station, award-winning with Sunday Buffet for £8.50), Hollywood Bowl (American-style restaurant at 4 High Street with burgers and ribs in a Hollywood-themed setting), Ossie's Fish and Chips (at 75 Richmond Road, known for the best fish and chips in the area), and Thai Won Mien (oriental restaurant at 59-61 High Street with noodles, duck and other oriental dishes).""",
    ]
    
    # Queries copied from data/queries.py
    LANDMARK_SEARCH_QUERIES = [
        "Find museums and galleries in Glasgow",
        "Show me restaurants serving Asian cuisine", 
        "What attractions can I see in Glasgow?",
        "Tell me about Monet's House",
        "Find places to eat in Gillingham"
    ]
    
    # Create mapping dictionary like the working source files
    QUERY_REFERENCE_ANSWERS = {
        query: answer for query, answer in zip(LANDMARK_SEARCH_QUERIES, LANDMARK_REFERENCE_ANSWERS)
    }
    
    # Prepare evaluation data with proper column names for Phoenix evaluators
    landmark_eval_data = []
    for _, row in landmark_results_df.iterrows():
        landmark_eval_data.append({
            "input": row["query"],
            "output": row["response"],
            "reference": QUERY_REFERENCE_ANSWERS.get(row["query"], "Reference answer not found"),
            "text": row["response"]  # For toxicity evaluation
        })
    
    # Ensure we only have 5 queries as intended
    if len(landmark_eval_data) > 5:
        logger.warning(f"Found {len(landmark_eval_data)} evaluation entries, limiting to first 5")
        landmark_eval_data = landmark_eval_data[:5]
    
    landmark_eval_df = pd.DataFrame(landmark_eval_data)
    logger.info(f"📊 Prepared {len(landmark_eval_df)} queries for Phoenix evaluation")
    
    try:
        # 1. Relevance Evaluation
        logger.info("🔍 Running Relevance Evaluation...")
        landmark_relevance_results = llm_classify(
            data=landmark_eval_df[["input", "reference"]],
            model=evaluator_llm,
            template=RAG_RELEVANCY_PROMPT_TEMPLATE,
            rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Relevance Evaluation Results:")
        # Extract labels from DataFrame results like the working script
        relevance_labels = landmark_relevance_results['label'].tolist() if 'label' in landmark_relevance_results.columns else []
        for i, result in enumerate(relevance_labels):
            # Add bounds checking to prevent IndexError
            if i < len(landmark_eval_data):
                query = landmark_eval_data[i]["input"]
            else:
                query = f"Query {i+1}"
            logger.info(f"   Query: {query}")
            logger.info(f"   Relevance: {result}")
            logger.info("   " + "-"*30)
        
        # 2. QA Evaluation
        logger.info("🔍 Running QA Evaluation...")
        landmark_qa_results = llm_classify(
            data=landmark_eval_df[["input", "output", "reference"]],
            model=evaluator_llm,
            template=QA_PROMPT_TEMPLATE,
            rails=list(QA_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ QA Evaluation Results:")
        # Extract labels from DataFrame results like the working script
        qa_labels = landmark_qa_results['label'].tolist() if 'label' in landmark_qa_results.columns else []
        for i, result in enumerate(qa_labels):
            # Add bounds checking to prevent IndexError
            if i < len(landmark_eval_data):
                query = landmark_eval_data[i]["input"]
            else:
                query = f"Query {i+1}"
            logger.info(f"   Query: {query}")
            logger.info(f"   QA Score: {result}")
            logger.info("   " + "-"*30)
        
        # 3. Hallucination Evaluation
        logger.info("🔍 Running Hallucination Evaluation...")
        landmark_hallucination_results = llm_classify(
            data=landmark_eval_df[["input", "reference", "output"]],
            model=evaluator_llm,
            template=HALLUCINATION_PROMPT_TEMPLATE,
            rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Hallucination Evaluation Results:")
        # Extract labels from DataFrame results like the working script
        hallucination_labels = landmark_hallucination_results['label'].tolist() if 'label' in landmark_hallucination_results.columns else []
        for i, result in enumerate(hallucination_labels):
            # Add bounds checking to prevent IndexError
            if i < len(landmark_eval_data):
                query = landmark_eval_data[i]["input"]
            else:
                query = f"Query {i+1}"
            logger.info(f"   Query: {query}")
            logger.info(f"   Hallucination: {result}")
            logger.info("   " + "-"*30)
        
        # 4. Toxicity Evaluation
        logger.info("🔍 Running Toxicity Evaluation...")
        landmark_toxicity_results = llm_classify(
            data=landmark_eval_df[["input"]],
            model=evaluator_llm,
            template=TOXICITY_PROMPT_TEMPLATE,
            rails=list(TOXICITY_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Toxicity Evaluation Results:")
        # Extract labels from DataFrame results like the working script
        toxicity_labels = landmark_toxicity_results['label'].tolist() if 'label' in landmark_toxicity_results.columns else []
        for i, result in enumerate(toxicity_labels):
            # Add bounds checking to prevent IndexError
            if i < len(landmark_eval_data):
                query = landmark_eval_data[i]["input"]
            else:
                query = f"Query {i+1}"
            logger.info(f"   Query: {query}")
            logger.info(f"   Toxicity: {result}")
            logger.info("   " + "-"*30)
        
        # Summary of all evaluations
        logger.info("📊 EVALUATION SUMMARY")
        logger.info("=" * 50)
        
        for i, query in enumerate([item["input"] for item in landmark_eval_data]):
            logger.info(f"Query {i+1}: {query}")
            # Extract labels from DataFrames using working script pattern
            try:
                relevance_labels = landmark_relevance_results['label'].tolist() if hasattr(landmark_relevance_results, 'columns') and 'label' in landmark_relevance_results.columns else []
                qa_labels = landmark_qa_results['label'].tolist() if hasattr(landmark_qa_results, 'columns') and 'label' in landmark_qa_results.columns else []
                hallucination_labels = landmark_hallucination_results['label'].tolist() if hasattr(landmark_hallucination_results, 'columns') and 'label' in landmark_hallucination_results.columns else []
                toxicity_labels = landmark_toxicity_results['label'].tolist() if hasattr(landmark_toxicity_results, 'columns') and 'label' in landmark_toxicity_results.columns else []
                
                relevance = relevance_labels[i] if i < len(relevance_labels) else "N/A"
                qa_score = qa_labels[i] if i < len(qa_labels) else "N/A"
                hallucination = hallucination_labels[i] if i < len(hallucination_labels) else "N/A"
                toxicity = toxicity_labels[i] if i < len(toxicity_labels) else "N/A"
                
                logger.info(f"  Relevance: {relevance}")
                logger.info(f"  QA Score: {qa_score}")
                logger.info(f"  Hallucination: {hallucination}")
                logger.info(f"  Toxicity: {toxicity}")
            except Exception as e:
                logger.warning(f"  Error accessing evaluation results: {e}")
            logger.info("  " + "-"*40)
        
        logger.info("✅ All Phoenix evaluations completed successfully!")
        
    except Exception as e:
        logger.exception(f"❌ Phoenix evaluation failed: {e}")
        logger.info("💡 This might be due to API rate limits or model availability")
        
else:
    if not ARIZE_AVAILABLE:
        logger.info("❌ Phoenix evaluations skipped - Arize dependencies not available")
    else:
        logger.info("❌ Phoenix evaluations skipped - No demo results to evaluate")


2025-09-04 13:02:43,290 - INFO - 🔍 Running comprehensive Phoenix evaluations...
2025-09-04 13:02:43,331 - INFO - 📊 Prepared 5 queries for Phoenix evaluation
2025-09-04 13:02:43,332 - INFO - 🔍 Running Relevance Evaluation...


llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-04 13:02:46,019 - INFO - ✅ Relevance Evaluation Results:
2025-09-04 13:02:46,020 - INFO -    Query: Find museums and galleries in Glasgow
2025-09-04 13:02:46,021 - INFO -    Relevance: relevant
2025-09-04 13:02:46,021 - INFO -    ------------------------------
2025-09-04 13:02:46,021 - INFO -    Query: Show me restaurants serving Asian cuisine
2025-09-04 13:02:46,022 - INFO -    Relevance: relevant
2025-09-04 13:02:46,023 - INFO -    ------------------------------
2025-09-04 13:02:46,023 - INFO -    Query: What attractions can I see in Glasgow?
2025-09-04 13:02:46,023 - INFO -    Relevance: relevant
2025-09-04 13:02:46,024 - INFO -    ------------------------------
2025-09-04 13:02:46,024 - INFO -    Query: Tell me about Monet's House
2025-09-04 13:02:46,025 - INFO -    Relevance: relevant
2025-09-04 13:02:46,025 - INFO -    ------------------------------
2025-09-04 13:02:46,025 - INFO -    Query: Find places to eat in Gillingham
2025-09-04 13:02:46,026 - INFO -    Relevance: r

llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-04 13:02:48,428 - INFO - ✅ QA Evaluation Results:
2025-09-04 13:02:48,429 - INFO -    Query: Find museums and galleries in Glasgow
2025-09-04 13:02:48,429 - INFO -    QA Score: incorrect
2025-09-04 13:02:48,429 - INFO -    ------------------------------
2025-09-04 13:02:48,430 - INFO -    Query: Show me restaurants serving Asian cuisine
2025-09-04 13:02:48,430 - INFO -    QA Score: incorrect
2025-09-04 13:02:48,430 - INFO -    ------------------------------
2025-09-04 13:02:48,430 - INFO -    Query: What attractions can I see in Glasgow?
2025-09-04 13:02:48,430 - INFO -    QA Score: incorrect
2025-09-04 13:02:48,431 - INFO -    ------------------------------
2025-09-04 13:02:48,431 - INFO -    Query: Tell me about Monet's House
2025-09-04 13:02:48,431 - INFO -    QA Score: correct
2025-09-04 13:02:48,432 - INFO -    ------------------------------
2025-09-04 13:02:48,432 - INFO -    Query: Find places to eat in Gillingham
2025-09-04 13:02:48,432 - INFO -    QA Score: incorrect
2

llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-04 13:02:50,672 - INFO - ✅ Hallucination Evaluation Results:
2025-09-04 13:02:50,674 - INFO -    Query: Find museums and galleries in Glasgow
2025-09-04 13:02:50,674 - INFO -    Hallucination: hallucinated
2025-09-04 13:02:50,675 - INFO -    ------------------------------
2025-09-04 13:02:50,675 - INFO -    Query: Show me restaurants serving Asian cuisine
2025-09-04 13:02:50,676 - INFO -    Hallucination: hallucinated
2025-09-04 13:02:50,677 - INFO -    ------------------------------
2025-09-04 13:02:50,678 - INFO -    Query: What attractions can I see in Glasgow?
2025-09-04 13:02:50,678 - INFO -    Hallucination: hallucinated
2025-09-04 13:02:50,679 - INFO -    ------------------------------
2025-09-04 13:02:50,680 - INFO -    Query: Tell me about Monet's House
2025-09-04 13:02:50,680 - INFO -    Hallucination: hallucinated
2025-09-04 13:02:50,680 - INFO -    ------------------------------
2025-09-04 13:02:50,681 - INFO -    Query: Find places to eat in Gillingham
2025-09-04 1

llm_classify |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

2025-09-04 13:02:52,556 - INFO - ✅ Toxicity Evaluation Results:
2025-09-04 13:02:52,557 - INFO -    Query: Find museums and galleries in Glasgow
2025-09-04 13:02:52,558 - INFO -    Toxicity: non-toxic
2025-09-04 13:02:52,558 - INFO -    ------------------------------
2025-09-04 13:02:52,559 - INFO -    Query: Show me restaurants serving Asian cuisine
2025-09-04 13:02:52,560 - INFO -    Toxicity: non-toxic
2025-09-04 13:02:52,560 - INFO -    ------------------------------
2025-09-04 13:02:52,560 - INFO -    Query: What attractions can I see in Glasgow?
2025-09-04 13:02:52,561 - INFO -    Toxicity: non-toxic
2025-09-04 13:02:52,561 - INFO -    ------------------------------
2025-09-04 13:02:52,561 - INFO -    Query: Tell me about Monet's House
2025-09-04 13:02:52,561 - INFO -    Toxicity: non-toxic
2025-09-04 13:02:52,561 - INFO -    ------------------------------
2025-09-04 13:02:52,562 - INFO -    Query: Find places to eat in Gillingham
2025-09-04 13:02:52,562 - INFO -    Toxicity: non

## Summary

This notebook demonstrates a complete landmark search agent implementation using:

1. **Agent Catalog Integration**: Using agentc to find tools and prompts
2. **LlamaIndex Framework**: ReAct agent pattern with semantic search capabilities
3. **Couchbase Vector Store**: Storing and searching landmark data from travel-sample bucket
4. **NVIDIA NIMs + Capella AI**: NVIDIA NIMs for LLM, Capella AI for embeddings
5. **Single Tool Architecture**: Focused on `search_landmarks` for landmark discovery
6. **Comprehensive Evaluation**: Phoenix-based evaluation with LlamaIndex instrumentation

The agent can handle various landmark-related queries including:
- Landmark search by location (Tokyo, London, Paris)
- Finding specific types of attractions (museums, parks, monuments)
- Cultural and historical site discovery
- Tourist attraction recommendations

## Phoenix Evaluation Metrics

The notebook demonstrates all four key Phoenix evaluation types:

1. **Relevance Evaluation**: Measures how relevant responses are to landmark queries
2. **QA Evaluation**: Assesses the quality and accuracy of landmark information
3. **Hallucination Detection**: Identifies fabricated or incorrect landmark information
4. **Toxicity Detection**: Screens for harmful or inappropriate content

Each evaluation provides:
- Binary or categorical labels (e.g., "relevant"/"irrelevant", "correct"/"incorrect")
- Detailed explanations of the evaluation reasoning
- Confidence scores for the assessments

## Key Features

This landmark search agent implementation:
- **Uses LlamaIndex**: Advanced RAG framework with ReAct agent pattern
- **Uses travel-sample bucket**: Leverages existing Couchbase landmark data
- **NVIDIA NIMs integration**: High-performance LLM inference
- **Capella AI embeddings**: High-quality vector embeddings for semantic search
- **OpenAI fallback**: Graceful fallback when Capella AI is unavailable
- **Single focused tool**: Simplified architecture with one search tool
- **Comprehensive evaluation**: Full Phoenix evaluation pipeline
- **LlamaIndex instrumentation**: Integrated observability and tracing

## Data Source

The agent uses landmark data from the `travel-sample.inventory.landmark` collection, which contains:
- Real landmark information with names, locations, and descriptions
- Structured data with address, city, country, and type information
- Rich text descriptions suitable for vector embedding
- Global coverage of tourist attractions and points of interest

## Architecture Differences

This landmark search agent differs from the other agents:
- **LlamaIndex** (not LangChain or LangGraph) - advanced RAG framework
- **NVIDIA NIMs LLM**: High-performance inference instead of OpenAI/Capella LLM
- **ReAct Pattern**: Built-in reasoning and action capabilities
- **Landmark-specific**: Optimized for tourism and travel use cases
- **Global Settings**: Uses LlamaIndex global settings for LLM and embeddings

For production use, consider:
- Setting up proper monitoring with Arize Phoenix
- Implementing comprehensive evaluation pipelines
- Adding error handling and retry logic
- Scaling the vector store for larger datasets
- Adding more sophisticated query understanding

## Usage Instructions

To run this notebook:
1. Set up the required environment variables (Couchbase connection, API keys)
2. Install dependencies: `pip install -r requirements.txt`
3. Ensure travel-sample bucket is available in your Couchbase cluster
4. Publish your agent catalog: `agentc index . && agentc publish`
5. Run the notebook cells sequentially

The agent will automatically load landmark data from travel-sample and create embeddings for semantic search capabilities. NVIDIA API key is required for LLM functionality.
